In [3]:
import torch
assert torch.cuda.is_available()

In [ ]:
e.g.
rfd3 design out_dir=inference_outputs/protein_binder/0 \
inputs=./protein_binder_design.json \
inference_sampler.step_scale=3 \
inference_sampler.gamma_0=0.2
總體來說(https://github.com/RosettaCommons/foundry/blob/production/models/rfd3/docs/input.md)
n_batches(1) — 每個 input key 要跑幾次「獨立的 diffusion batch/跑幾次推論」
diffusion_batch_size(8) — 每個batch裡面要同時生成多少設計
最後會生成n_batches * diffusion_batch_size

specification({}) — 腦死的講法就是input參數

inference_sampler.num_timesteps(200) — backward的步數 較少(50~100)粗糙 較大(>300) 精細/平滑
inference_sampler.step_scale(1.5) — 反向 diffusion 每一步更新的「步幅大小」
                                    實作上是:x_t-1 = x_t + step_scale*model_predicted_update
                                    較大(2.0) 收斂快, 多樣性下降, 保守 較小(0.8~1.0) 變化大, 不穩定

cfg(先跳過)



In [ ]:
json檔案設定

input: (如果contig+length的話就一定要出現) PDB/CIF檔案
atom_array: 建議不用 跳過
contig: 分段, 告訴說我要生成多長的結構, 那段當成target
unindex: 指定哪些殘基不是完全固定，模型可以對這些殘基進行結構調整或重新生成
length: 總體的長度
ligand: 這個蛋白結構中有的配體名稱，RFdiffusion 會考慮它們在結構中
dialect(2) : parser的設定一般來說不用動它
select_fixed_atoms: True
                    所有從 input 檔案（PDB / CIF）經由 contig 拉進來的原子
                    在 diffusion 中位置完全固定, 不會被去噪＆更新
                    partial-diffusion/scaffold/binding partner
                    False
                    diffusion 中可以自由移動(等於denovo)
select_unfixed_sequence(True): 哪裡可以重新設計
select_buried/select_partially_buried/select_exposed
select_hbond_donor/select_hbond_acceptor
select_hotspots: binder-design recommand
redesign_motif_sidechains(bool)

symmetry

ori_token(list[float]): 手動指定 ORI token 的 3D 座標/覆寫自動計算的原點/通常 COM 落在 ORI token ± ~5Å-> 我要讓binder固定/放在某一區
infer_ori_strategy('com'/'hotspot'): 沒有給定ori_token的時候就會根據中心(com)或指定的hotspot('hotspot', 10A)自動計算

plddt_enhanced
is_non_loopy: True的話loop會比較少

partial_t(float): 從已有結構加噪音在修回來一部分(loop remodeling/局部重排backbone refinement), t這裡是在幾何上的擾動幅度而非步數

In [ ]:
contig 設定
A40-60,70,A120-170,A203,\0,B3-45,60-80


* A40-60
    * 從 input 的 A chain 取 residue 40–60
    * backbone + 座標被保留（可搭配 select_fixed_atoms 決定是否可動）

* 70
    * de novo 設計 70 residues
    * 接到 A60 的 C-terminal
    * 長度固定 70

* A120-170
    * 在 de novo 後面接回 input 的 A120–A170
    * 自動建立 peptide bond

* A203
    * 在 A170 之後接 A203

* A171–A202 不存在

* \0
    * chain break
    * A203 與下一段（B3）之間沒有 peptide bond

* B3-B45
    * 從 input B chain 拿 B3–B45

* 60-80
    * de novo 設計一段 60–80 residues
    * 接在 B45 後面
    * 最終長度隨機在這個範圍內

* 注意點
    * 位置與 chain label
        * 帶 chain label → input
        * 不帶 chain label → de novo

    * 連續性
        * 如果想打斷鏈，用 \0

* 範圍隨機性
    * de novo range → 最終長度隨機在範圍內


In [ ]:
"M0255_1mg5_unfixed": {
        "input": "./input_pdbs/M0255_1mg5.pdb", //原始 PDB 檔案，作為模板
        "ligand": "NAI,ACT", //這個蛋白結構中有的配體名稱，RFdiffusion 會考慮它們在結構中
        "unindex": "A108,A139,A152,A156", //指定 哪些殘基不是完全固定，模型可以對這些殘基進行結構調整或重新生成
        "length": "180-200", //生成或設計的目標蛋白長度範圍
        "select_fixed_atoms": { //指定 在(就算在unindex中仍)要固定的原子。
            "A108": "ND2,CG", //例如 A108 殘基的 ND2,CG 原子會被固定，其他原子可能可以被模型改動。
            "A139": "OG,CB,CA",
            "A152": "OH,CZ",
            "A156": "NZ,CE,CD",
            "ACT": "OXT",
            "NAI": ""
        }
    },

In [ ]:
"partial_diffusion": {
        "input": "./input_pdbs/7v11.pdb",
        "ligand": "OQO",
        "partial_t": 15.0, //部分擴散的程度，通常用於「局部擴散」生成，模型只對部分結構做修改
        "contig": "A431", //指定要設計或修改的片段區間
        "unindex": "A572-573",
        "select_fixed_atoms": {
            "A431": "TIP",
            "A572": "BKBN",
            "A573": "BKBN"
        }
    },

In [ ]:
"dsDNA_basic": {
        "input": "./input_pdbs/1bna.pdb",
        "contig": "A1-10,/0,B15-24,/0,120-130", //指定要設計或修改的片段區間
        "length": "140-150",
        "ori_token": [24,20,10],
        "is_non_loopy": true
    }

In [ ]:
# binder design
"pdl1": {
        "dialect": 2,
        "infer_ori_strategy": "hotspots", //
        "input": "input_pdbs/5o45_cropped.pdb", //你要設計 binder 去結合的蛋白。
        "contig": "50-120,/0,A17-131", //binder 會以你在 select_hotspots 指定的原子（A56、A115、A123）為「對齊中心」，生成 binder。
        "select_hotspots": {
            "A56": "CG,OH", //hostpot不要變動
            "A115": "CG,SD",
            "A123": "CD2,OH",
       },
        "is_non_loopy": true
    }
rfd3 design out_dir=inference_outputs/protein_binder/0 \
inputs=./protein_binder_design.json \
inference_sampler.step_scale=3 \
inference_sampler.gamma_0=0.2

In [1]:
from lightning.fabric import seed_everything
from rfd3.engine import RFD3InferenceConfig, RFD3InferenceEngine
from pathlib import Path
from atomworks.io.utils.visualize import view
# Set seed for reproducibility
seed_everything(52)
#簡單說就是把json檔寫在RFD3InferenceConfig裡面而已
# Configure RFD3 inference
config = RFD3InferenceConfig(
    specification={
        #'length': 80,  # de novo 80aa, no backbone, no contig,
        "input": '/home/theodore/PycharmProjects/PythonProject/input_pdbs/5o45_cropped.pdb',
        "infer_ori_strategy": "hotspots",
        "contig": "50-120,/0,A17-131",
        "select_hotspots":{
            "A56": ["CG", "OH"],
            "A115": ["CG", "SD"],
            "A123": ["CD2", "OH"],
        },
        "is_non_loopy": True,
        'extra': {},  # We are not using any extra specifications here.
    },
    diffusion_batch_size=2,  # Generate 2 structures per batch
)

# Initialize engine and run generation
model = RFD3InferenceEngine(**config)
outputs = model.run(
    inputs=None,      # None for unconditional generation
    out_dir=None,     # None to return in memory (no file output)
    n_batches=1,      # Generate 1 batch
)

Environment variable CCD_MIRROR_PATH not set. Will not be able to use function requiring this variable. To set it you may:
  (1) add the line 'export VAR_NAME=path/to/variable' to your .bashrc or .zshrc file
  (2) set it in your current shell with 'export VAR_NAME=path/to/variable'
  (3) write it to a .env file in the root of the atomworks.io repository
Environment variable PDB_MIRROR_PATH not set. Will not be able to use function requiring this variable. To set it you may:
  (1) add the line 'export VAR_NAME=path/to/variable' to your .bashrc or .zshrc file
  (2) set it in your current shell with 'export VAR_NAME=path/to/variable'
  (3) write it to a .env file in the root of the atomworks.io repository
13:02:35 DEBUG transforms: Debug mode is on
Seed set to 52
Using bfloat16 Automatic Mixed Precision (AMP)
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` whi

In [2]:
print(outputs.keys())
# Inspect RFD3 outputs and extract the generated backbone
for idx, data in outputs.items():
    print(f"Batch {idx}: {len(data)} structure(s)")
    print(f"  Output type: {type(data[0]).__name__}")
    print(f"  AtomArray: {data[0].atom_array}")
# Extract the first generated backbone for downstream use
first_key = next(iter(outputs.keys()))
atom_array = outputs[first_key][0].atom_array
view(atom_array)


dict_keys(['_0'])
Batch _0: 2 structure(s)
  Output type: RFD3Output
  AtomArray:     A       1  GLY N      N        17.693  -16.715   -1.906
    A       1  GLY CA     C        16.677  -15.963   -2.394
    A       1  GLY C      C        15.287  -16.447   -2.805
    A       1  GLY O      O        15.214  -17.089   -3.831
    A       2  ALA N      N        14.272  -16.170   -2.005
    A       2  ALA CA     C        12.913  -16.445   -2.388
    A       2  ALA C      C        12.498  -15.645   -3.616
    A       2  ALA O      O        12.857  -14.473   -3.754
    A       2  ALA CB     C        11.974  -16.190   -1.221
    A       3  ALA N      N        11.780  -16.202   -4.542
    A       3  ALA CA     C        11.276  -15.593   -5.750
    A       3  ALA C      C        10.309  -14.478   -5.454
    A       3  ALA O      O        10.219  -13.525   -6.217
    A       3  ALA CB     C        10.703  -16.669   -6.659
    A       4  ALA N      N         9.597  -14.539   -4.390
    A       4  ALA

3Dmol.js failed to load for some reason. Please check your browser console for error messages.